PincoeVectorDB Practicle

In [1]:
import torch
# Set device and torch dtype for the model on the GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [3]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")

Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
%pip install langchain
%pip install pinecone-client
%pip install pypdf
%pip install openai
%pip install tiktoken

In [4]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.embeddings import OpenAIEmbeddings
#from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
import os

In [5]:
# passing pdf data
loader=PyPDFDirectoryLoader("pdfs")
data=loader.load()

In [6]:
data[0]

Document(page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing w

# Recursively split by character
This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is ["\n\n", "\n", " ", ""]. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=20
)

In [8]:
text_chunks = text_splitter.split_documents(data)

In [9]:
print(text_chunks[0].page_content)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto


In [10]:
len(text_chunks)

91

In [11]:
print(text_chunks[90].page_content)

just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>Figure 5: Many of the attention heads exhibit behaviour that seems related to the structure of the
sentence. We give two such examples above, from two different heads from the encoder self-attention
at layer 5 of 6. The heads clearly learned to perform different tasks.
15


### we will import pinecode and store embessing data in it

###### Before getting started, decide whether to use serverless or pod-based index.

In [17]:

import os
use_serverless = True



#### Creating an Index
Now the data is ready, we can set up our index to store it.

We begin by initializing our connection to Pinecone. To do this we need a free API key.

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index.

In [25]:
print(spec)

ServerlessSpec(cloud='gcp-starter', region='us-central1')


In [28]:
index_name = 'testing'

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

##### Listing indexes
Using the legacy API, the list_indexes operation returns an array of index names and no other information. Using the new API, the operation now fetches a complete description of each index. The data returned on each index is equivalent to what you get back from the describe_index operation.

In [41]:
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'testing-kr1pfok.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'testing',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [46]:
pc.describe_index("testing")

{'dimension': 1536,
 'host': 'testing-kr1pfok.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'testing',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [60]:
import os

# Set the environment variable
os.environ["OPENAI_API_KEY"] = "swcWYAZD"

# Now you can access it without encountering a KeyError
api_key = os.environ["OPENAI_API_KEY"]

embedding1 = OpenAIEmbeddings()

In [78]:
#from pinecone import ServerlessSpec, PodSpec
from langchain_pinecone import Pinecone
YOUR_API_KEY='77286cd406'
environment = 'gcp-starter'

# check if index already exists (it shouldn't if this is first time)
if index_name not in Pinecone.list_indexes().names():
    # if does not exist, create index
    Pinecone.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine'
    )
# connect to index
index = Pinecone.Index(index_name)
# view index stats
index.describe_index_stats()

AttributeError: type object 'Pinecone' has no attribute 'list_indexes'

In [79]:


Pinecone.list_indexes()

#docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embedding1, index_name=index_name)

AttributeError: type object 'Pinecone' has no attribute 'list_indexes'

In [ ]:
docsearch